<a href="https://colab.research.google.com/github/FahimShahriarAnik/NLP/blob/main/%20Exploring-N-gram-Language-Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

### N-gram Language Model
In this assignment, you will build several n-gram models. For submission, you will need to first ***make a copy of this notebook*** (File-> save a copy to drive). Next, fill out each of the functions for each task. Make sure to run each cell and check if the output is correct or not. Finally, submit a pdf (File->print). Note, you should not use any library for the code other than the ones already imported.

In [ ]:
# import and download corpus
import nltk
from nltk.corpus import gutenberg
import numpy as np

nltk.download('gutenberg')
nltk.download('punkt')

# getting a list of words from this book
caesar = gutenberg.words('shakespeare-caesar.txt')
# convert all to lower case
caesar = [l.lower() for l in caesar]
# print first ten words here
print(caesar[210:220])

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[',', 'i', 'can', 'mend', 'you', 'mur', '.', 'what', 'mean', "'"]


#### Task 1: Write functions to count unique occurences and co-occurence for words next to each other only. (Points: 5)



In [ ]:
def count_unique_occurences(list_of_words):
  list_of_lists = []
  # list_of_lists should be a lit of tuples (a, b) where a is the word and b is the count
  # replace the list here with your code
  #list_of_lists = [('julius', 5), ('ceasur', 3)]

  word_count = {}
  for word in list_of_words:
      if word in word_count:
          word_count[word] += 1
      else:
          word_count[word] = 1

  # Convert the dictionary to a list of tuples
  list_of_lists = list(word_count.items())

  return list_of_lists
#count_unique_occurences(caesar)

In [ ]:
def count_co_occurences(list_of_words):
  LST_DICT = []
  # LST_DICT should be a lit of tuples (a, b) where a is the word1:word2 and b is the count
  # replace the list here with your code
  #LST_DICT = [('julius:ceasur', 5), ('i:am', 3)]
  co_occurrence_count = {}
  for i in range(len(list_of_words) - 1):
      pair = f"{list_of_words[i]}:{list_of_words[i + 1]}"
      if pair in co_occurrence_count:
          co_occurrence_count[pair] += 1
      else:
          co_occurrence_count[pair] = 1

  # Convert the dictionary to a list of tuples
  LST_DICT = list(co_occurrence_count.items())

  return LST_DICT

In [ ]:
# print first five occurences
occurences = count_unique_occurences(caesar)
_ = [print(l) for l in occurences[:5]]

print(len(occurences))

('[', 3)
('the', 579)
('tragedie', 2)
('of', 354)
('julius', 1)
3032


In [ ]:
# print first five co-occurences
co_occurences = count_co_occurences(caesar)
_ = [print(l) for l in co_occurences[:5]]

('[:the', 1)
('the:tragedie', 2)
('tragedie:of', 2)
('of:julius', 1)
('julius:caesar', 1)


#### Task 2: write functions to compute unigram and bigram proability (Points: 5)

In [ ]:
def occurences_probability(list_of_counts):
  list_of_dicts = []
  for word, count in list_of_counts:
    list_of_dicts.append((word, str(count/len(list_of_counts))))

  return list_of_dicts

In [ ]:
def occurences_of_one_word(word):
  for w, count in occurences:
    if w == word:
      return count
  return 1

def co_occurences_probability(list_of_counts):
  list_of_dicts = []

  for word, count in list_of_counts:
    words= word.split(":")
    list_of_dicts.append((word, str((count+1)/ (occurences_of_one_word(words[1]) + len(list_of_counts) ) ))) # implemented the add-1 estimate formula

  return list_of_dicts

#co_occurences_probability(co_occurences)

In [ ]:
# print first five occurences
occ_prob = occurences_probability(occurences)
_ = [print(l) for l in occ_prob[:5]]
print(len(occ_prob))

('[', '0.0009894459102902375')
('the', '0.19096306068601582')
('tragedie', '0.0006596306068601583')
('of', '0.11675461741424802')
('julius', '0.00032981530343007914')
3032


In [ ]:
# print first five co-occurences probabilities
co_occ_prob = co_occurences_probability(co_occurences)
_ = [print(l) for l in co_occ_prob[:5]]

('[:the', '0.00013410218586562962')
('the:tragedie', '0.0002092487968194183')
('tragedie:of', '0.0002042344611614133')
('of:julius', '0.00013950892857142856')
('julius:caesar', '0.0001376936316695353')


#### Task 3: Find probability of a sentence using unigram and bigram model (points 5)

In [ ]:
def unigram_sent_probability(sent, prob_list):
  prob = 0.1
  words = sent.split()
  for word in words:
    for w, p in prob_list:
      if w == word:
        prob *= float(p)

  return prob

In [ ]:
def bigram_sent_probability(sent, prob_list):
  prob = 0.2
  # compute probability
  # here sent will be a string that you will have to split
  words = sent.split()
  for i in range(len(words) - 1):
    for word, p in prob_list:
      pair = f"{words[i]}:{words[i + 1]}"
      if word == pair:
        prob *= float(p)

  return prob

In [ ]:
sent = "i can mend you"
u_prob = unigram_sent_probability(sent, occ_prob)
b_prob = bigram_sent_probability(sent, co_occ_prob)
print("Unigram probability for", sent, "is:", u_prob)
print("Bigram probability for", sent, "is:", b_prob)

Unigram probability for i can mend you is: 9.370672832607475e-09
Bigram probability for i can mend you is: 2.375836078964408e-12


#### Task 4: Predict the most probable next word (Points 5)

In [ ]:
def uni_predict_next_words(sent, prob_list, n=1):
  words = sent.split()
  for w,p in prob_list:
    if w in words:
      prob_list.remove((w,p))


  output_words = []
  ## calculating next best prob word, appending it to returning list and removing it from prob_list
  for i in range(n):
    max_p = -1
    for word, p in prob_list:
      if float(p) > max_p:
        max_p = float(p)
        max_p_word = word
        #print(max_p_word)
    output_words.append(max_p_word)
    prob_list.remove((max_p_word, str(max_p)))

  return output_words

In [ ]:
def bi_predict_next_words(sent, prob_list, n=1):
  # go through all combination of words and find the next n words with highest probability and return the list
  # replace this with your code
  #word = ['I']

  output_words = []
  words = sent.split()

  last_word = words[len(words)-1]

  for i in range(n):
    max_p = -1
    next_word = ''

    for word, p in prob_list:
      if word.split(":")[0] == last_word:
        if float(p) > max_p:
          max_p = float(p)
          max_p_word = word
          next_word = word.split(":")[1]
    output_words.append(next_word)
    last_word = next_word
    prob_list.remove((max_p_word, str(max_p)))

  return output_words

In [ ]:
u_next_words = uni_predict_next_words(sent, occ_prob, n=10)
b_next_words = bi_predict_next_words(sent, co_occ_prob, n=10)
print("Unigram-based next best word for", sent, "is:", u_next_words)
print("Bigram-based next best word for", sent, "is:", b_next_words)

Unigram-based next best word for i can mend you is: [',', '.', 'and', 'the', ':', 'to', "'", 'of', '?', 'that']
Bigram-based next best word for i can mend you is: ['are', 'not', ',', 'and', 'i', 'will', ',', 'i', 'am', 'i']


#### Task 5: Interpolated Probability. (Points 5)

In [ ]:
def linearly_interpolated_sent_probability(sent, lam1, prob_list_u, lam2, prob_list_b):
  # here instead of uni or bigram, you will use linear interpolation to find the probability of the sentence
  # lambda values are lam1 for unigram and lam2 for bigram
  # replace this line with your code
  prob = 0.0
  prob += lam1 * unigram_sent_probability(sent, prob_list_u)
  prob += lam2 * bigram_sent_probability(sent, prob_list_b)

  return prob

In [ ]:
# lets see how probability changes with lambda values
for i in range(1,10):
  l_prob = linearly_interpolated_sent_probability(sent, i/10, occ_prob, 1-(i)/10, co_occ_prob)
  print("With lamda ", i/10, "%.1f" % (1-(i)/10), "interpolated probability for", sent, "is:", l_prob)

With lamda  0.1 0.9 interpolated probability for i can mend you is: 6.596308206854054e-06
With lamda  0.2 0.8 interpolated probability for i can mend you is: 1.319261403787203e-05
With lamda  0.3 0.7 interpolated probability for i can mend you is: 1.9788919868890004e-05
With lamda  0.4 0.6 interpolated probability for i can mend you is: 2.638522569990798e-05
With lamda  0.5 0.5 interpolated probability for i can mend you is: 3.298153153092595e-05
With lamda  0.6 0.4 interpolated probability for i can mend you is: 3.957783736194393e-05
With lamda  0.7 0.3 interpolated probability for i can mend you is: 4.61741431929619e-05
With lamda  0.8 0.2 interpolated probability for i can mend you is: 5.277044902397988e-05
With lamda  0.9 0.1 interpolated probability for i can mend you is: 5.936675485499785e-05
